<a href="https://colab.research.google.com/github/dbizzaro/NotebooksKRL/blob/main/SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAT Solving

## DIMACS CNF FORMAT

The **dimacs** format is a textual representation of a formula in CNF.

It uses positive integers to represent propositional variables, and their negative for the corresponding negated variable.

A dimacs file begins with a header line: $\text{ p cnf }  |\text{variables}| \,|\text{clauses}|$.

Clauses are encoded as a sequence of decimal numbers. Usually, each clause is listed on a separate line, using spaces between each of the literals, and a final zero.

As an example, the formula $(x \vee y \vee \neg z) \wedge (\neg y \vee z)$ could be encoded in this way:

\begin{align}
& \text{p} \ \ \text{cnf} \  \ 3 \ \ 2 \\
& 1 \ \  2 \  \ -3 \ \ 0 \\
& -2 \ \ 3 \ \ 0
\end{align}

where $x$ corresponds to $1$, $y$ to $2$, and $z$ to $3$.

### Encoding

For representing any problem, we need to define a function
(encoding) that maps each variable $p\in\mathcal{P}$ into a natural number:
$$
Encode:\mathcal{P}\longrightarrow[n]
$$
where $[n]:=\{1,2,\dots,n\}$ and $n=|\mathcal{P}|$.  
The $Decode$ function from $[n]$ to $\mathcal{P}$ is the inverse
of $Encode$:
$$
Decode:[n]\longrightarrow\mathcal{P}, \quad Decode(i)=Encode^{-1}(i)
$$

To generate the lines in the body of a dimacs file, we use $Encode(p)$ for any positive literal $p$ in the clause, and $-Encode(p)$ for a negative literal.

**EXERCISE:** write in dimacs the following clauses: $$\{\{A,\neg B,\neg D\},\{\neg A,\neg B,\neg C\},\{\neg A,C,\neg
  D\},\{\neg A,B,C\}\}$$

In [146]:
# Encoding:
# A -> 1
# B -> 2
# C -> 3
# D -> 4

# Write solution
dimacs_str = """
p cnf 4 4
1 -2 -4 0
-1 -2 -3 0
-1 3 -4 0
-1 2 3 0
"""

### SAT solvers

We can call the SAT solver on the encoding that we have
generated. The sat solver can return either UNSATISFIABLE
or SATISFIABLE. In the latter case, we can ask the solver to
return a model, codified as a sequence
of positive and negative integers:
$$
r = [\pm 1, \pm 2,\dots, \pm n]
$$
where $n=|\mathcal{P}|$. The list
encodes the following interpretation:
$$
\mathcal{I}_r(p) =
\begin{cases}
True & \mbox{if } Encode(p)\in r \\
False & \mbox{if } -Encode(p)\in r
\end{cases}
$$

In [147]:
# Use a SAT solver from pysat python library
!pip install python-sat

In [148]:
from pysat.formula import CNF
from pysat.solvers import Minisat22

# Parse the DIMACS string
cnf = CNF(from_string=dimacs_str)

# Solve using PySAT
solver = Minisat22()
solver.append_formula(cnf)

if solver.solve():
    print("SATISFIABLE")
    model = solver.get_model()
    print("Solution:", model)  # Get a satisfying assignment
else:
    print("UNSATISFIABLE")

SATISFIABLE
Solution: [-1, -2, -3, -4]


If we want to obtain additional models, we have to add to the initial
set of clauses that fact that we want a model different from $\mathcal{I}_r$.
A model of $\Gamma$ is different from $\mathcal{I}_r$ if at least one
proposition takes a truth value different from the one assigned by
$\mathcal{I}_r$.  We can therefore add to $\Gamma$ the clause
$$
\bigvee_{\mathcal{I}_r(p)=True}\neg p \vee
\bigvee_{\mathcal{I}_r(p)=False}p
$$
which can be codified as the sequence $-r = [-i \mid i\in r]$.

In [149]:
while solver.solve():
    model = solver.get_model()
    print(model)
    solver.add_clause([-i for i in model])

[-1, -2, -3, -4]
[-1, -2, 3, -4]
[-1, -2, 3, 4]
[1, -2, 3, 4]
[-1, -2, -3, 4]
[-1, 2, -3, -4]
[-1, 2, 3, -4]
[1, -2, 3, -4]
[1, 2, -3, -4]


In [150]:
# need to reinstantiate the solver
solver = Minisat22()
solver.append_formula(cnf)

# Alternative enumeration of models
all_models = list(solver.enum_models())
for model in all_models:
    print(model)

[-1, -2, -3, -4]
[-1, 2, -3, -4]
[-1, 2, 3, -4]
[1, 2, -3, -4]
[-1, -2, 3, -4]
[1, -2, 3, -4]
[1, -2, 3, 4]
[-1, -2, 3, 4]
[-1, -2, -3, 4]


## NumberMind
The game NumberMind is a variant of the well known game MasterMind.
Instead of colored pegs, you have to guess a secret sequence of
digits. After each guess, you’re only told in how many places you have
guessed the correct digit.

So, if the sequence was 1234 and you
guessed 2036, you’d be told that you have one correct digit; however,
you would NOT be told that you also have another digit in the wrong
place. As another example, the following table shows some feedbacks for the secret sequence **39542**:

| **Guess**  | **Feedback** |
|------------|-------------|
| 903**42**  | 2 correct   |
| 70794      | 0 correct   |
| **39**458  | 2 correct   |
| **3**4109  | 1 correct   |
| 51**54**5  | 2 correct   |
| 12**5**31  | 1 correct   |

How can we use a SAT solver to generate guesses that take into
account the feedbacks?



The relevant atomic propositions for the NumberMind world are those expressing that a certain number is in a certain position of the secret sequence. Therefore, for every position $p\in\{0,\dots,p_{\max}\}$ and every digit $d\in\{0,\dots,9\}$, we consider the following propositional variable:
    \begin{align*}
      \text{In}(d,p):= &&& \text{there is a digit } d \mbox{ in position } p
    \end{align*}

The second step is to provide encoding and decoding functions for the
dimacs format. Simple encoding/decoding functions are the
following:
    \begin{align*}
      Encode(\text{In}(d,p)) & = p\cdot10+ d + 1 \\
      Decode(k) & = \text{In}(d,p) & \mbox{ where } d & = (k-1)\mod{10} \\
     && p & = (k-1)\div 10
    \end{align*}



In [151]:
def encode(d, p):
  """Encodes the proposition 'In(digit, position)' into a unique integer."""
  return p * 10 + d + 1

def decode(encoded_value):
  """Decodes an encoded value back into the 'digit' and 'position'."""
  encoded_value -= 1
  d = encoded_value % 10
  p = encoded_value // 10
  return d, p

def feedback(secret_solution, guess):
  """Computes the number of correct digits in a guess."""
  return sum(1 for p, d in enumerate(guess) if d == secret_solution[p])

In the next step, we have to formalize the constraints of the game.
In NumberMind, constrains come from two sources:
*  First, in the sequence, one only one digit per position is admitted.
* The second source of constraints is the feedback provided when the player proposes a guess (so these constraints are added every time the player proposes a guess).

Both of these type of constraints are cardinality constraints, and they can be encoded in the following way:

* At every position there is exactly one digit:
    \begin{align}
      \bigwedge_{p=0}^{p_{\max}}\bigvee_{d=0}^{9}\text{In}(d,p)\wedge\bigwedge_{p=0}^{p_{\max}}\bigwedge_{0\leq
      d < d'\leq 9}\neg\text{In}(d,p)\vee \neg\text{In}(d',p)
    \end{align}
* There are $k$ correct digits in a guess made of the sequence $[d_1, \dots, d_{p_{\max}}]$:
    \begin{align}
      \bigwedge_{\substack{I\subseteq G \\
      |I|=p_{\max}+1-k+1}}\bigvee_{\text{In}(d,p)\in I} \text{In}(d,p)\wedge
      \bigwedge_{\substack{I\subseteq G \\ |I| = k+1}}\bigvee_{\text{In}(d,p)\in
      I}\neg \text{In}(d,p)
    \end{align}
  where $G=\{\text{In}(d_1,1),\text{In}(d_2,2),\dots,\text{In}(d_{p_{\max}},p_{\max})\}$

In [153]:
from itertools import combinations
from pysat.solvers import Minisat22

solver = Minisat22()
secret_sequence = [3, 9, 5, 4, 2]
p_max = len(secret_sequence)

# Encode the first constraints
for p in range(p_max):
  solver.add_clause([encode(d, p) for d in range(10)])
  for d1, d2 in combinations(range(10), 2):
      solver.add_clause([-encode(d1, p), -encode(d2, p)])

def add_feedback_constraint(guess, k):
  """Adds a feedback constraint to the solver for guess having k correct digits."""
  variables = [encode(d, p) for p, d in enumerate(guess)]
  for subset in combinations(variables, len(variables) - k + 1):
    solver.add_clause(list(subset))
  for subset in combinations(variables, k + 1):
    solver.add_clause([-lit for lit in subset])

In [154]:
def solve(solver, secret_solution):
  p_max = len(secret_solution)
  correct_guesses = 0
  while correct_guesses < p_max:
    if not solver.solve():
      raise ValueError("No solution found!")
    model = solver.get_model()
    guess = [next(d for d in range(10) if model[encode(d, p) - 1] > 0) for p in range(p_max)]
    correct_guesses = feedback(secret_solution, guess)
    print(f"Guess: {''.join(map(str, guess))}, Correct: {correct_guesses}")
    if correct_guesses == p_max:
      print("Solution found!")
      return guess
    add_feedback_constraint(guess, correct_guesses)


solve(solver, secret_sequence)

Guess: 00000, Correct: 0
Guess: 11111, Correct: 0
Guess: 22222, Correct: 1
Guess: 23333, Correct: 0
Guess: 94442, Correct: 2
Guess: 94529, Correct: 1
Guess: 89542, Correct: 4
Guess: 79542, Correct: 4
Guess: 59542, Correct: 4
Guess: 49542, Correct: 4
Guess: 69542, Correct: 4
Guess: 39542, Correct: 5
Solution found!


[3, 9, 5, 4, 2]